In [112]:
import os, math
import pandas as pd
import pandapower as pp
import numpy as np

data_dir = 'Modified_116_LV_CSV'

In [113]:
# Source data
source_df = pd.read_csv(os.path.join(data_dir,'Source.csv'), skiprows=1, sep='=')
source_dict = {i:float(row.iloc[0].split()[0]) for i, row in source_df.iterrows()}
print(source_dict)

source_dict['ISC1'] = source_dict['ISC1'] / 1000
source_dict['ISC3'] = source_dict['ISC3'] / 1000

# Transformer data
trafo_df = pd.read_csv(os.path.join(data_dir,'Transformer.csv'), skiprows=1, sep=';')
trafo_dict = trafo_df.iloc[0].to_dict()
print(trafo_dict)

{'Voltage': 11.0, 'pu': 1.05, 'ISC3': 3000.0, 'ISC1': 5.0}


{'Name': 'TR1', ' phases': 3, ' bus1': 'SourceBus', ' bus2': 1, ' kV_pri': 11, ' kV_sec': 0.416, ' MVA': 0.8, ' Conn_pri': ' Delta', ' Conn_sec': ' Wye', ' %XHL': 4, '% resistance': 0.4}


In [114]:
s_sc_mva = np.sqrt(3) * source_dict['Voltage'] * source_dict['ISC3'] #MVA

net = pp.create_empty_network()

bus_map = {}

hv_bus = pp.create_bus(net, name=trafo_dict[' bus1'], vn_kv=source_dict['Voltage'], type="b")
lv_bus = pp.create_bus(net, name=trafo_dict[' bus2'], vn_kv=trafo_dict[' kV_sec'], type="b")

bus_map[trafo_dict[' bus1']] = hv_bus
bus_map[trafo_dict[' bus2']] = lv_bus

pp.create_ext_grid(
    net, bus=hv_bus, vm_pu=source_dict['pu'],
    s_sc_max_mva=s_sc_mva,
    rx_max=0.1, rx_min=None,
    # max_p_mw=None, min_p_mw=None,
    # max_q_mvar=None, min_q_mvar=None, index=None,
    r0x0_max=0.1, x0x_max=1.0
)

pp.create_transformer_from_parameters(
    net=net, hv_bus=hv_bus, lv_bus=lv_bus,
    sn_mva=trafo_dict[' MVA'],
    vn_hv_kv=trafo_dict[' kV_pri'],
    vn_lv_kv=trafo_dict[' kV_sec'],
    vk_percent=trafo_dict[' %XHL'],
    vkr_percent=trafo_dict['% resistance'],
    pfe_kw=0.0, i0_percent=0.0, #shift_degree=0.0, 
    name=trafo_dict['Name']
)

pp.create_shunt(
    net, bus=lv_bus,
    q_mvar=-0.01, p_mw=10e-3,
    vn_kv=trafo_dict[' kV_sec'],
    name="trafo_lv_shunt"
)

np.int64(0)

In [115]:
lines_df = pd.read_excel(os.path.join(data_dir, "Lines.xlsx"), skiprows=1)
lines_df['Length'] = lines_df['Length'] / 1000 # m to km
lines_df['Units'] = 'km'
lines_df.head()

lineCodes_df = pd.read_csv(os.path.join(data_dir, "LineCodes.csv"), skiprows=1, sep=';')

all_bus_ids = np.unique(lines_df[['Bus1', 'Bus2']].values.ravel('K'))
for id in all_bus_ids:
    id = math.floor(id)
    if id not in bus_map.keys():
        bus = pp.create_bus(net, name=id, vn_kv=trafo_dict[' kV_sec'], type="b")
        bus_map[id] = bus

In [116]:
lineCodes_df['max_i_ka'] = np.ones(len(lineCodes_df))

In [117]:
full_line_df = lines_df.merge(lineCodes_df, left_on="LineCode", right_on="Name", how="left")
# full_line_df.head(10)

net.line.drop(net.line.index, inplace=True)
for _, line in full_line_df.iterrows():
    # print(line)

    if (line['C0'] == 0): line['C0'] = 200 # nF/km
    if (line['C1'] == 0): line['C1'] = 200 # nF/km
    
    pp.create_line_from_parameters(
        net, from_bus = bus_map[math.floor(line['Bus1'])],
        to_bus = bus_map[math.floor(line['Bus2'])],
        length_km = line['Length'],
        r_ohm_per_km=line["R1"], r0_ohm_per_km=line["R0"],
        x_ohm_per_km=line["X1"], x0_ohm_per_km=line["X0"],
        c_nf_per_km=line["C1"], c0_nf_per_km=line["C0"],
        max_i_ka=line["max_i_ka"],
        name=line["Name_x"]
    )

In [118]:
suspicious_lines = net.line[
    (np.isclose(net.line["length_km"], 0, rtol=1e-2)) |
    (np.isclose(net.line["r_ohm_per_km"], 0, rtol=1e-2)) |
    (np.isclose(net.line["x_ohm_per_km"], 0, rtol=1e-2))
]
suspicious_lines

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo,r0_ohm_per_km,x0_ohm_per_km,c0_nf_per_km,g0_us_per_km


In [119]:
net.trafo["vector_group"] = 'Dyn'
net.trafo["vk0_percent"] = net.trafo["vk_percent"]
net.trafo["vkr0_percent"] = net.trafo["vkr_percent"]
net.trafo["mag0_percent"] = 100
net.trafo["mag0_rx"] = 0
net.trafo["si0_hv_partial"] = 0.9

In [120]:
import pandas as pd

def export_results(file_path, net, init='auto', max_iteration=100, tolerance_mva=1e-8):
    """
    Export all result DataFrames (attributes of net starting with 'res_') to an Excel file.
    
    Parameters:
    - file_path: str, path to the Excel file to save the results.
    - net: object containing result DataFrames as attributes starting with 'res_'.
    """

    try:
        pp.runpp_3ph(
            net, init=init,
            max_iteration=max_iteration,
            tolerance_mva=tolerance_mva,
            calc_voltage_angles=True,
            v_debug=True
        )
    except Exception as e:
        return e  # or raise e if you want the exception to propagate

    with pd.ExcelWriter(file_path) as writer:
        for attr in dir(net):
            if attr.startswith("res_"):
                df = getattr(net, attr)
                if isinstance(df, pd.DataFrame) and not df.empty:
                    sheet_name = attr[:31]  # Excel sheet names must be <= 31 characters
                    df.to_excel(writer, sheet_name=sheet_name)
    return True

def debug_result(net, init='auto', max_iteration=100, tolerance_mva=1e-8):
    try:
        pp.runpp_3ph(
            net, init=init,
            max_iteration=max_iteration,
            tolerance_mva=tolerance_mva,
            calc_voltage_angles=True,
            v_debug=True
        )
    except Exception as e:
        return False
    return True

In [121]:
pp.diagnostic(net)



_____________ PANDAPOWER DIAGNOSTIC TOOL _____________ 


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------


 --------

_____________ END OF PANDAPOWER DIAGNOSTIC _____________ 


{'implausible_impedance_values': [{'line': [1,
    2,
    3,
    4,
    5,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    19,
    21,
    22,
    23,
    24,
    25,
    26,
    29,
    30,
    31,
    33,
    35,
    36,
    37,
    39,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    51,
    54,
    55,
    56,
    57,
    58,
    61,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    71,
    72,
    74,
    76,
    79,
    80,
    82,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    93,
    94,
    97,
    98,
    99,
    101,
    102,
    103,
    105,
    106,
    107,
    108,
    109,
    110,
    111,
    112,
    113,
    114]}]}

In [122]:
export_results("basic_network_results.xlsx", net)

True

In [123]:
# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))
sample_loads = loads_df.sample(15)

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in sample_loads.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']

print(net.asymmetric_load)
export_results("batch_network_results.xlsx", net)

      name  bus  p_a_mw  q_a_mvar  p_b_mw  q_b_mvar  p_c_mw  q_c_mvar  sn_mva  \
0   LOAD46  103   0.001  0.000329   0.000  0.000000   0.000  0.000000     NaN   
1   LOAD41   93   0.000  0.000000   0.001  0.000329   0.000  0.000000     NaN   
2   LOAD19   46   0.000  0.000000   0.000  0.000000   0.001  0.000329     NaN   
3   LOAD15   39   0.000  0.000000   0.001  0.000329   0.000  0.000000     NaN   
4   LOAD12   33   0.000  0.000000   0.000  0.000000   0.001  0.000329     NaN   
5   LOAD49  107   0.001  0.000329   0.000  0.000000   0.000  0.000000     NaN   
6   LOAD55  116   0.001  0.000329   0.000  0.000000   0.000  0.000000     NaN   
7   LOAD54  115   0.001  0.000329   0.000  0.000000   0.000  0.000000     NaN   
8   LOAD30   67   0.001  0.000329   0.000  0.000000   0.000  0.000000     NaN   
9   LOAD16   40   0.000  0.000000   0.000  0.000000   0.001  0.000329     NaN   
10  LOAD27   62   0.000  0.000000   0.000  0.000000   0.001  0.000329     NaN   
11  LOAD48  106   0.001  0.0

pandapower.auxiliary.LoadflowNotConverged('Power Flow nr did not converge after 100 iterations!')

In [124]:
N = 500
batch = list(range(5, 51, 5))
for b in batch:
    success = 0
    for i in range(N):
        sample_loads = loads_df.sample(b)

        net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
        load_map = {}
        for _, row in sample_loads.iterrows():
            # print(_, '->', row)
            bus_id = math.floor(row['Bus'])
            pa = qa = pb = qb = pc = qc = 0
            if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
                if row['phases']=='A':
                    pa = row['kW']
                    qa = row['kW'] * np.tan(np.arccos(row['PF']))
                elif row['phases']=='B':
                    pb = row['kW']
                    qb = row['kW'] * np.tan(np.arccos(row['PF']))
                else: # row['phases']=='C'
                    pc = row['kW']
                    qc = row['kW'] * np.tan(np.arccos(row['PF']))
                load = pp.create_asymmetric_load(
                    net, bus=bus_map[bus_id],
                    p_a_mw=pa, q_a_mvar=qa,
                    p_b_mw=pb, q_b_mvar=qb,
                    p_c_mw=pc, q_c_mvar=qc,
                    name=row['Name']
                )
                load_map[load] = row['Yearly']

        if debug_result(net, init='flat'): success += 1
    print(f"[{b}]\tConvergence rate = {100 * success / N} %")

c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

[5]	Convergence rate = 100.0 %


c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

KeyboardInterrupt: 

In [ ]:
# Load by load:
load_problem = []

for _, row in loads_df.iterrows():
    net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
    
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
    if not debug_result(net, init='flat'): load_problem.append(row['Name'])

load_problem

c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

[]

In [ ]:
# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in loads_df.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']


55
Bus 0:  {'trafo': [0], 'bus': [np.uint32(1)], 'ext_grid': [0]}
Bus 1:  {'shunt': [0], 'trafo': [0], 'bus': [np.uint32(0), 2], 'line': [0]}
Bus 2:  {'bus': [1, 3, 4], 'line': [0, 1, 2]}
Bus 3:  {'bus': [2, 10, 5], 'line': [1, 3, 4]}
Bus 4:  {'bus': [8, 2, 6], 'line': [2, 5, 6]}
Bus 5:  {'asymmetric_load': [0], 'bus': [3], 'line': [3]}
Bus 6:  {'bus': [4, 13, 7], 'line': [8, 5, 7]}
Bus 7:  {'asymmetric_load': [1], 'bus': [6], 'line': [7]}
Bus 8:  {'bus': [9, 4, 14], 'line': [9, 10, 6]}
Bus 9:  {'bus': [8, 11, 12], 'line': [9, 11, 12]}
Bus 10:  {'asymmetric_load': [2], 'bus': [3], 'line': [4]}
Bus 11:  {'asymmetric_load': [3], 'bus': [9], 'line': [11]}
Bus 12:  {'asymmetric_load': [4], 'bus': [9], 'line': [12]}
Bus 13:  {'asymmetric_load': [5], 'bus': [6], 'line': [8]}
Bus 14:  {'bus': [8, 19, 15], 'line': [10, 13, 14]}
Bus 15:  {'bus': [16, 28, 14], 'line': [16, 13, 15]}
Bus 16:  {'bus': [17, 37, 15], 'line': [17, 18, 15]}
Bus 17:  {'bus': [16, 18, 31], 'line': [17, 19, 20]}
Bus 18:  

In [134]:
import pandapower.networks as pn
offpeak_net = pn.ieee_european_lv_asymmetric('on_peak_566')

In [137]:
offpeak_net.asymmetric_load

,name,bus,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar,sn_mva,scaling,in_service,type,description,sernum,for_name,chr_name
0,LOAD1,34,0.000574,0.000189,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
1,LOAD2,47,0.000000,0.000000,0.000044,1.446210e-08,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
2,LOAD3,70,0.000114,0.000037,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
3,LOAD4,73,0.000231,0.000076,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
4,LOAD5,74,0.000035,0.000012,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
5,LOAD6,83,0.000000,0.000000,0.000395,1.298302e-07,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
6,LOAD7,178,0.000000,0.000000,0.000055,1.807763e-08,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
7,LOAD8,208,0.000000,0.000000,0.000000,0.000000e+00,0.002836,9.321481e-07,NaN,1.0,True,wye,,,,
8,LOAD9,225,0.000131,0.000043,0.000000,0.000000e+00,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,
9,LOAD10,248,0.000000,0.000000,0.002798,9.196581e-07,0.000000,0.000000e+00,NaN,1.0,True,wye,,,,


In [ ]:
def hc_violation(net):
    # pp.runpp(net)
    return any([
        net.res_trafo_3ph.loading_percent.max() > 100, # 110
        net.res_line_3ph.loading_a_percent.max() > 100, # 110
        net.res_line_3ph.loading_b_percent.max() > 100,
        net.res_line_3ph.loading_c_percent.max() > 100,
        net.res_bus_3ph.vm_a_pu.max() >= 1.05, # 1.10
        net.res_bus_3ph.vm_b_pu.max() >= 1.05,
        net.res_bus_3ph.vm_c_pu.max() >= 1.05,
        net.res_bus_3ph.vm_a_pu.min() <= 0.95, # 0.85
        net.res_bus_3ph.vm_b_pu.min() <= 0.95,
        net.res_bus_3ph.vm_c_pu.min() <= 0.95
    ])